# MATRIX ARRAY

In [2]:
%load_ext memory_profiler
%load_ext snakeviz
%load_ext cython
%load_ext autoreload
%autoreload 2

from IPython.core import debugger
ist = debugger.set_trace

In [3]:
from py.typyMagics import *
ipy = get_ipython()
ipy.register_magics(typyMagics)

In [4]:
import sys
sys.path.insert(0,'../')

## DEFINE


See for fast inverse https://stackoverflow.com/q/11972102

In [5]:
%%run_and_write ../typyPRISM/core/MatrixArray.py
#!python
from typyPRISM.core.Space import Space
from itertools import product
import numpy as np

class MatrixArray:
    '''A container for creating and interacting with arrays of matrices
    
    The primary data structure of MatrixArray is simply a 3D Numpy array 
    with the first dimension accessing each individual matrix in the array
    and the last two dimenions corresponding to the vertical and horizontal 
    index of each matrix element.
    
    The terminology *column* is used to refer to the set of values from
    all matrices in the array at a given matrix index pair. In Numpy slicing 
    parlance::
    
        column_11 = numpy_array[:,1,1]
        column_12 = numpy_array[:,1,2]
    
    
    Attributes
    ----------
    rank: int
        Number of rows/cols of each (square) matrix. For PRISM theory, this 
        also equal to the number of site types.
        
    length: int
        Number of matrices in array. For PRISM theory, this corresponds to
        the number of grid points in real- and Fourier-space i.e. Domain.size.
        
    types: list, *optional*
        List of semantic types that can be used to reference data via the 
        getByTypes method. These types will be output by the itercolumn
        method as well. If not supplied, integer types will be generated.
        
    data: float np.ndarray, size (length,rank,rank)
        Interface for specifying the MatrixArray data directly. If not given,
        all matrices will be set to zero. 
    
    space: typyPRISM.Space
        Enumerated value tracking whether the array represents real or Fourier
        spaced data. As we will be transferring arrays to and from these spaces,
        it's important for safety that we track this.
    '''
    # __slots__ = ('rank','length','data','space')
    
    SpaceError = "Attempting MatrixArray math in non-matching spaces"
    
    def __init__(self,length,rank,data=None,space=None,types=None):
                    
        if data is None:
            self.data = np.zeros((length,rank,rank))
            self.rank = rank
            self.length = length
        else:
            self.data = data
            self.rank = data.shape[1]
            self.length = data.shape[0]
        
        if types is None:
            self.types = list(range(self.rank))
        else:
            assert len(types)==self.rank
            self.types
        
        if space is None:
            self.space = Space.Real
        else:
            self.space = space
            
    def __repr__(self):
        return '<MatrixArray rank:{:d} length:{:d}>'.format(self.rank,self.length)
    
    def itercolumn(self):
        for i,j in product(range(self.rank),range(self.rank)):
            if i<=j: #upper triangle condition
                type1 = self.types[i]
                type2 = self.types[j]
                yield (i,j),(type1,type2),self.data[:,i,j]
            
    def __setitem__(self,key,val):
        '''Column setter 
        
        Assumes all matrices are symmetric and enforces symmetry by
        setting both off diagonal elements. 
        '''
        index1,index2 = key
        self.data[:,index1,index2] = val
        if not (index1 == index2):
            self.data[:,index2,index1] = val
        
    def __getitem__(self,key):
        '''Column getter'''
        index1,index2 = key
        return self.data[:,index1,index2]
    
    def getByTypes(self,type1,type2):
        '''Column getter via supplied types
        
        .. warning::
        
            This getter should not be used in performance critical code
            as it has to look up the index of the semantic types.
        
        '''
        index1 = self.types.index(type1)
        index2 = self.types.index(type2)
        return self.data[:,index1,index2]
    
    def __truediv__(self,other):
        '''Scalar or elementwise division'''
        if type(other) is MatrixArray:
            assert self.space == other.space,MatrixArray.SpaceError
            data = self.data / other.data
        else:
            data = self.data / other
        return MatrixArray(length=self.length,rank=self.rank,data=data,space=self.space)
    
    def __itruediv__(self,other):
        '''Scalar or elementwise division'''
        if type(other) is MatrixArray:
            assert self.space == other.space,MatrixArray.SpaceError
            self.data /= other.data
        else:
            self.data /= other
        return self
    
    # def __rmul__(self,other):
    #     '''Scalar or elementwise multiplication'''
    #     return self.__mul__(other)
        
    def __mul__(self,other):
        '''Scalar or elementwise multiplication'''
        if type(other) is MatrixArray:
            assert self.space == other.space,MatrixArray.SpaceError
            data = self.data * other.data
        else:
            data = self.data * other
        return MatrixArray(length=self.length,rank=self.rank,data=data,space=self.space)
    
    def __imul__(self,other):
        '''Scalar or elementwise multiplication'''
        if type(other) is MatrixArray:
            assert self.space == other.space,MatrixArray.SpaceError
            self.data *= other.data
        else:
            self.data *= other
        return self
    
    # def __radd__(self,other):
    #     '''Scalar or elementwise addition'''
    #     return self.__add__(other)
            
    def __add__(self,other):
        if type(other) is MatrixArray:
            assert self.space == other.space,MatrixArray.SpaceError
            data = self.data + other.data
        else:
            data = self.data + other
        return MatrixArray(length=self.length,rank=self.rank,data=data,space=self.space)
    
    def __iadd__(self,other):
        if type(other) is MatrixArray:
            assert self.space == other.space,MatrixArray.SpaceError
            self.data += other.data
        else:
            self.data += other
        return self
            
    # def __rsub__(self,other):
    #     '''Scalar or elementwise subtraction'''
    #     return self.__sub__(other)
    
    def __sub__(self,other):
        if type(other) is MatrixArray:
            assert self.space == other.space,MatrixArray.SpaceError
            data = self.data - other.data
        else:
            data = self.data - other
        return MatrixArray(length=self.length,rank=self.rank,data=data,space=self.space)
    
    def __isub__(self,other):
        if type(other) is MatrixArray:
            assert self.space == other.space,MatrixArray.SpaceError
            self.data -= other.data
        else:
            self.data -= other
        return self
            
    def invert(self,inplace=False):
        '''Perform matrix inversion on all matrices in the MatrixArray
        
        Parameters
        ----------
        inplace: bool
            If False, a new MatrixArray is returned, otherwise just
            update the internal data.
        '''
        if inplace:
            data = self.data
        else:
            data = np.copy(self.data)
            
        data = np.linalg.inv(self.data)
            
        if inplace:
            self.data = data
            return self
        else:
            return MatrixArray(rank=self.rank,length=self.length,data=data,space=self.space)
        
    def dot(self,other,inplace=False):
        ''' Matrix multiplication for each matrix in two MatrixArrays
        
        Parameters
        ----------
        other: object, MatrixArray
            Must be an object of MatrixArray type of the same length
            and dimension
            
        inplace: bool
            If False, a new MatrixArray is returned, otherwise just
            update the internal data.
        
        '''
        if inplace:
            self.data = np.einsum('lij,ljk->lik', self.data, other.data)
            return self
        else:
            data = np.einsum('lij,ljk->lik', self.data, other.data)
            return MatrixArray(length=self.length,rank=self.rank,data=data,space=self.space)
        
    def __matmul__(self,other):
        assert self.space == other.space,MatrixArray.SpaceError
        return self.dot(other,inplace=False)
        
    def __imatmul__(self,other):
        assert self.space == other.space,MatrixArray.SpaceError
        return self.dot(other,inplace=True)
        
        


Overwriting ../typyPRISM/core/MatrixArray.py


In [12]:
%%run_and_write ../typyPRISM/core/IdentityMatrixArray.py
#!python
from typyPRISM.core.MatrixArray import MatrixArray
from typyPRISM.core.Space import Space
import numpy as np

class IdentityMatrixArray(MatrixArray):
    '''Specialization of MatrixArray for Identity Matrices '''
    __slots__ = ('rank','length','data','space')
    
    def __init__(self,length,rank,data=None,space=None,types=None):
        self.rank = rank
        self.length = length
        
        if types is None:
            self.types = list(range(rank))
        else:
            assert len(types)==rank
            self.types
        
        if data is None:
            self.data = np.zeros((length,rank,rank))
            for i in range(rank):
                self.data[:,i,i] = 1.0
        else:
            self.data = data
            
        if space is None:
            self.space = Space.Real
        else:
            self.space = space
        


Overwriting ../typyPRISM/core/IdentityMatrixArray.py


## TEST

In [13]:
%%run_and_write ../typyPRISM/test/MatrixArray_TestCase.py
#!python
from typyPRISM.core.MatrixArray import MatrixArray
import numpy as np
import unittest

class MatrixArray_TestCase(unittest.TestCase): 
    def test_assign(self):
        '''Can we create and assign values?'''
        length = 100
        rank = 8
        MA = MatrixArray(length=length,rank=rank)
        
        # Make sure the array starts as all zeros
        array = np.zeros((length,rank,rank))
        np.testing.assert_array_almost_equal(MA.data,array)
        
        #  Test assignment (especially off diagonal)
        array = np.zeros((length,rank,rank))
        MA[1,1] = np.ones(length)
        MA[1,2] = np.ones(length)*3.0
        array[:,1,1] = np.ones(length)
        array[:,1,2] = np.ones(length)*3.0
        array[:,2,1] = np.ones(length)*3.0
        np.testing.assert_array_almost_equal(MA.data,array)
        
    def test_div(self):
        '''Can we truediv and itruediv?'''
        
        length = 100
        rank = 3
        (MA1,MA2),(array1,array2) = self.set_up_test_arrays(length,rank)
        
        MA2[0,2] = np.ones(length)*2.0
        MA2[0,1] = np.ones(length)*3.0
        MA2[1,2] = np.ones(length)*5.0
        MA2[0,0] += 1.0
        
        array2[:,0,2] = array2[:,2,0] = np.ones(length)*2.0
        array2[:,0,1] = array2[:,1,0] = np.ones(length)*3.0
        array2[:,1,2] = array2[:,2,1] = np.ones(length)*5.0
        array2[:,0,0] += 1.0
        
        ## Test Add
        MA3 = MA1 / MA2
        array3 = array1 / array2
        MA3 = MA3 / 542.345
        array3 = array3 / 542.345
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        np.testing.assert_array_almost_equal(MA3.data,array3)
        
        ## Test iAdd
        MA3 /= MA2
        array3 /= array2
        MA3 /= 324
        array3 /= 324
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        np.testing.assert_array_almost_equal(MA3.data,array3)
        
    def set_up_test_arrays(self,length=100,rank=3):
        ''' Helper for set up arrays to test math'''
        MA1 = MatrixArray(length=length,rank=rank)
        MA1[0,0] = np.ones(length)*2.0
        MA1[1,1] = np.ones(length)
        MA1[2,2] = np.ones(length)*3.0
        MA1[1,2] = np.ones(length)*3.0
        
        MA2 = MatrixArray(length=length,rank=rank)
        MA2[0,0] = np.arange(length)
        MA2[1,1] = np.ones(length)*2.0
        MA2[2,2] = np.ones(length)*-3.0
        
        array1 = np.zeros((length,rank,rank))
        array1[:,0,0] = np.ones(length)*2.0
        array1[:,1,1] = np.ones(length)
        array1[:,2,2] = np.ones(length)*3.0
        array1[:,1,2] = np.ones(length)*3.0
        array1[:,2,1] = np.ones(length)*3.0
        
        array2 = np.zeros((length,rank,rank))
        array2[:,0,0] = np.arange(length)
        array2[:,1,1] = np.ones(length)*2.0
        array2[:,2,2] = np.ones(length)*-3.0
        return (MA1,MA2),(array1,array2)
        
    def test_add(self):
        '''Can we add and iadd?'''
        
        length = 100
        rank = 3
        (MA1,MA2),(array1,array2) = self.set_up_test_arrays(length,rank)
        
        ## Test Add
        MA3 = MA1 + MA2
        array3 = array1 + array2
        MA3 = MA3 + 2.53
        array3 = array3 + 2.53
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        np.testing.assert_array_almost_equal(MA3.data,array3)
        
        ## Test iAdd
        MA3 += MA2
        MA3 += 435.43
        array3 += array2
        array3 += 435.43
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        np.testing.assert_array_almost_equal(MA3.data,array3)
        
    def test_sub(self):
        '''Can we sub and isub?'''
        
        length = 100
        rank = 3
        (MA1,MA2),(array1,array2) = self.set_up_test_arrays(length,rank)
        
        ## Test Add
        MA3 = MA1 - MA2
        array3 = array1 - array2
        MA3 = MA3 - 852.32
        array3 = array3 - 852.32
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        np.testing.assert_array_almost_equal(MA3.data,array3)
        
        ## Test iAdd
        MA3 -= MA2
        array3 -= array2
        MA3 -= 2
        array3 -= 2
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        np.testing.assert_array_almost_equal(MA3.data,array3)
        
    def test_mul(self):
        '''Can we mul and imul?'''
        
        length = 100
        rank = 3
        (MA1,MA2),(array1,array2) = self.set_up_test_arrays(length,rank)
        
        ## Test Add
        MA3 = MA1 * MA2
        array3 = array1 * array2
        MA3 = MA3 * 542.345
        array3 = array3 * 542.345
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        np.testing.assert_array_almost_equal(MA3.data,array3)
        
        ## Test iAdd
        MA3 *= MA2
        array3 *= array2
        MA3 *= 324
        array3 *= 324
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        np.testing.assert_array_almost_equal(MA3.data,array3)
        
    def test_invert(self):
        '''Can we matrix invert?'''
        
        length = 100
        rank = 3
        (MA1,_),(array1,_) = self.set_up_test_arrays(length,rank)
        
        MA2 = MA1.invert(inplace=False)
        
        array2 = np.empty_like(array1)
        for i in range(length):
            array2[i] = np.linalg.inv(array1[i])
        
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        
        MA1.invert(inplace=True)
        np.testing.assert_array_almost_equal(MA1.data,MA2.data)
        
    def test_dot(self):
        '''Can we matrix multiply?'''
        
        length = 100
        rank = 3
        (MA1,MA2),(array1,array2) = self.set_up_test_arrays(length,rank)
        
        # MA3 = MA1.dot(MA2,inplace=False)
        MA3 = MA1 @ MA2
        
        array3 = np.empty_like(array1)
        for i in range(length):
            array3[i] = np.dot(array1[i],array2[i])
        
        np.testing.assert_array_almost_equal(MA1.data,array1)
        np.testing.assert_array_almost_equal(MA2.data,array2)
        np.testing.assert_array_almost_equal(MA3.data,array3)
        
        MA1.dot(MA2,inplace=True)
        np.testing.assert_array_almost_equal(MA1.data,MA3.data)
        
    def test_itercolumn(self):
        ''' Can we iterate over the columns?'''
        length = 100
        rank = 5
        (MA1,_),(array1,_) = self.set_up_test_arrays(length,rank)
        
        ncols = 0
        for (i,j),(t1,t2),col in MA1.itercolumn():
            with self.subTest(i=i,j=j):
                np.testing.assert_array_almost_equal(col,array1[:,i,j])
            ncols+=1
        self.assertEqual(ncols,rank*(rank+1)//2)
                
    def test_itercolumn_assign(self):
        ''' Can we assign as we iterate over the columns?'''
        length = 100
        rank = 4
        (MA1,_),(array1,_) = self.set_up_test_arrays(length,rank)
        
        ncols = 0
        for (i,j),(t1,t2),col in MA1.itercolumn():
            MA1[i,j] = np.ones(length)*i + j/2.0
            array1[:,i,j] = np.ones(length)*i + j/2.0
            array1[:,j,i] = np.ones(length)*i + j/2.0
            ncols += 1 
            
        np.testing.assert_array_almost_equal(MA1.data,array1)
        self.assertEqual(ncols,rank*(rank+1)//2)
            
    def test_getbytypes(self):
        ''' Can we access the data by semantic types?'''
        length = 100
        rank = 3
        types = ['A','B','C']
        (MA1,_),(array1,_) = self.set_up_test_arrays(length,rank)
        
        MA1.types = types
        
        np.testing.assert_array_almost_equal(MA1.getByTypes('A','A'),MA1[0,0])
        np.testing.assert_array_almost_equal(MA1.getByTypes('A','B'),MA1[0,1])
        np.testing.assert_array_almost_equal(MA1.getByTypes('A','C'),MA1[0,2])
        np.testing.assert_array_almost_equal(MA1.getByTypes('B','A'),MA1[1,0])
        np.testing.assert_array_almost_equal(MA1.getByTypes('B','B'),MA1[1,1])
        np.testing.assert_array_almost_equal(MA1.getByTypes('B','C'),MA1[1,2])
        np.testing.assert_array_almost_equal(MA1.getByTypes('C','A'),MA1[2,0])
        np.testing.assert_array_almost_equal(MA1.getByTypes('C','B'),MA1[2,1])
        np.testing.assert_array_almost_equal(MA1.getByTypes('C','C'),MA1[2,2])
        
            
        
        
    

Overwriting ../typyPRISM/test/MatrixArray_TestCase.py


In [10]:
import unittest
suite = unittest.TestLoader().loadTestsFromTestCase(MatrixArray_TestCase)
unittest.TextTestRunner(verbosity=2).run(suite)

test_add (__main__.MatrixArray_TestCase)
Can we add and iadd? ... ok
test_assign (__main__.MatrixArray_TestCase)
Can we create and assign values? ... ok
test_div (__main__.MatrixArray_TestCase)
Can we truediv and itruediv? ... ok
test_dot (__main__.MatrixArray_TestCase)
Can we matrix multiply? ... ok
test_getbytypes (__main__.MatrixArray_TestCase)
Can we access the data by semantic types? ... ok
test_invert (__main__.MatrixArray_TestCase)
Can we matrix invert? ... ok
test_itercolumn (__main__.MatrixArray_TestCase)
Can we iterate over the columns? ... ok
test_itercolumn_assign (__main__.MatrixArray_TestCase)
Can we assign as we iterate over the columns? ... ok
test_mul (__main__.MatrixArray_TestCase)
Can we mul and imul? ... ok
test_sub (__main__.MatrixArray_TestCase)
Can we sub and isub? ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.047s

OK


<unittest.runner.TextTestResult run=10 errors=0 failures=0>